<a href="https://colab.research.google.com/github/cimbelli/Inland_Navigable_Waterway/blob/main/s1_danube_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install geojson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
!pip install earthengine-api


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import ee
#import time
import matplotlib
import numpy as np
import requests
#import geemap

import os
import time
import geojson
import datetime
import json

import folium

In [15]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
           
def download(url, tipo, name):
    response = requests.get(url)
    
    if tipo == "raster":
        imgref = os.path.join(name + ".tif")
        with open(imgref, 'wb') as fd:
            fd.write(response.content)
    else:
        vec = os.path.join(name + ".geojson")
        with open(vec, 'wb') as fd:
            fd.write(response.content)
            
    return None

def download_raster(img, name):
    url = img.getDownloadUrl({
        'bands': ['VV'],
        'region': bb,
        'scale' : 10,
        'format': 'GEO_TIFF',
        'maxPixels': 834607840,
        'formatOptions': {
            'cloudOptimized': True
          }
    })
    download(url,"raster", name)
    return None

In [16]:
#start_time = time.time()
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()
maxPixels: 6362493000

sDate0 = '2021-01-01'    #last 6 months 2021
fDate = '2021-12-31'

image_viz_params = {
    'bands': ['VV'],
    'min': 0.0,
    'max': 1.0,
    #'gamma': [0.95, 1.1, 1]
}

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=M41USS9xkLUKSCgbjlaRRKZIuYv3ZB8ytT9u7hzrvdw&tc=5AkbHBbgDsqmBOb8xfe6a8q3PWj2oqkQY_WXns3ifAU&cc=vqI-TI598cQstLmjaoMjPXfvlATOcBp5WvVW19Pcmp4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJqeX4DW7V0eCr0rc2qmIy1P3Oy4trtDCy6M1sfr5dObWDcO_dIkNZA

Successfully saved authorization token.


In [17]:
f = open('waterway_poly4.geojson')
data1 = json.load(f)
tot_len_rivers = 0
tot_len_canals = 0
for i in range(len(data1['features'])):
    #print(data1['features'][i]['properties']['id'])
    
    coords = data1['features'][i]['geometry']['coordinates']
    [j.pop(2) for j in coords[0][0]]
    id1 = data1['features'][i]['properties']['id']
    type1 = data1['features'][i]['properties']['type']
    lenkm1 = data1['features'][i]['properties']['len_km']
    aoi = ee.Geometry.Polygon(coords[0][0])
    
    #if id1 > 20:
        #print(id1)

    imgVV = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')   #S1_GRD_FLOAT
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) 
            .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .select('VV')
            .filterDate(sDate0, fDate)
            .filterBounds(aoi)
            #.reduce(ee.Reducer.percentile([50]))
            )

    #imgVV_fixed = imgVV.median().clip(aoi).toUint8()#.focalMax(6).focalMin(6).gte(1)
    #imgVV_all = imgVV.first().clip(aoi).toUint8()#.focalMax(6).focalMin(6).gte(1)
    #im = imgVV_all.subtract(imgVV_fixed)

    imgVV_fixed = imgVV.max().clip(aoi).toUint8()
    im = imgVV_fixed

    center = list(aoi.centroid().getInfo().values())[1][1], list(aoi.centroid().getInfo().values())[1][0]

    ####### map
    #map1 = folium.Map(location=center, zoom_start=10)
    #folium.Map.add_ee_layer = add_ee_layer
    #map1.add_ee_layer(im, image_viz_params, 'false color composite')
    #display(map1)

    classes = im.reduceToVectors(
        **{
        'reducer': ee.Reducer.countEvery(),
        'geometry': aoi.getInfo(),
        'scale': 10,
        'maxPixels': 100000000
    })

    vessels = (ee.FeatureCollection(classes).filter(ee.Filter.gt('count', 15)).filter(ee.Filter.lt('count', 10000)))
    vessels_centroids = vessels.map(lambda f: f.centroid(10))

    #vessels_centroids1 = vessels_centroids.intersection(aoi)
    vessels_centroids1 = vessels_centroids.filter(ee.Filter.bounds(aoi))
    n_vessels = vessels_centroids1.size().getInfo()


    print("there are", n_vessels, "vessels", "in the", type1, "with id", id1, "and lenght", lenkm1, "km")
    
    if n_vessels >0:
        downloadUrl1 = vessels_centroids1.getDownloadURL(**{'filetype': 'GEOJSON'})
        download(downloadUrl1, "waterpoly_", "waterpoly_"+ str(id1))
        if type1 == "river":
            tot_len_rivers += lenkm1
        else:
            tot_len_canals += lenkm1

print()
print(tot_len_rivers, "kms of Navigable rivers and lakes")
print(tot_len_canals, "kms of Navigable canals")
print("Total of", tot_len_rivers + tot_len_canals, "kms of Inland waterways")

FileNotFoundError: ignored